In [19]:
import codecs

def slashescape(err):
    """ codecs error handler. err is UnicodeDecode instance. return
    a tuple with a replacement for the unencodable part of the input
    and a position where encoding should continue"""
    #print err, dir(err), err.start, err.end, err.object[:err.start]
    thebyte = err.object[err.start:err.end]
    repl = u'\\x'+hex(ord(thebyte))[2:]
    return (repl, err.end)

codecs.register_error('slashescape', slashescape)

In [29]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdftypes import resolve1
import pyperclip

with open('paper/pdf_comments/Perrenoud et al. - 2020 - Les musiciennes de « musiques actuelles » en Suiss.fdf', 'rb') as fdf_file:
    parser = PDFParser(fdf_file)
    doc = PDFDocument(parser)

    fields = resolve1(doc.catalog['FDF'])['Annots']
    previous_page = 0
    base_page = 103
    content_string = ''
    
    for i in fields:
        field = resolve1(i)
        #print(field)
        if str(field['Subtype']) == "/'Highlight'":
            try:
                current_page = field['Page']
                    
                content = field['Contents']
                #print(content)
                content_decoded = content.decode('utf-8', 'slashescape')
                # replacing byte code of return carriage by white space
                content_decoded = content_decoded.replace('\r', ' ')
                # replacing byte codes of quotation marks by regular quotation marks
                content_decoded = content_decoded.replace('\\x8e', '"')
                content_decoded = content_decoded.replace('\\x8f', "'")
                content_decoded = content_decoded.replace('\\x90', "'")
                content_decoded = content_decoded.replace('\\x8d', '"')
                content_decoded = content_decoded.replace('\\xe9', 'é')
                content_decoded = content_decoded.replace('\\xe8', 'è')
                content_decoded = content_decoded.replace('\\xea', 'ê')
                content_decoded = content_decoded.replace('\\xc9', 'É')
                content_decoded = content_decoded.replace('\\xf4', 'ô')
                content_decoded = content_decoded.replace('\\xe0', 'à')
                content_decoded = content_decoded.replace('\\xe2', 'â')
                content_decoded = content_decoded.replace('\\xc0', 'À')
                content_decoded = content_decoded.replace('\\xfc', 'ü')
                content_decoded = content_decoded.replace('\\xf9', 'ù')
                content_decoded = content_decoded.replace('\\xe7', 'ç')
                content_decoded = content_decoded.replace('\\xee', 'î')
                content_decoded = content_decoded.replace('\\xab ', '"')
                content_decoded = content_decoded.replace(' \\xbb', '"')
                content_decoded = content_decoded.replace('\\x85', '-')
                content_decoded = content_decoded.replace('\\x80', '\n•')
                
                content_string += content_decoded + '\n' + '\t' + str(base_page + current_page) + '\n'
            except KeyError:
                pass
    
    #print(content_string)
    pyperclip.copy(content_string)
    print(len(content_string))

4315
